# Implementation of baseline using Artificial Neural Network with Original + Win-loss + feature vectors feature set and BSA

Necessary files: logistic_regression_functions.py, ann_functions.py, testing_functions.py, win_loss_functions.py, feature_vectors_functions, BRA.csv, A_BSA.csv, H_BSA.csv, A_BSA_before_threshold.csv, H_BSA_before_threshold.csv

In [ ]:
import logistic_regression_functions
import ann_functions
import testing_functions
import win_loss_functions
import feature_vectors_functions
import pandas as pd
from datetime import datetime
import requests
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import *
from tqdm import tqdm

The baseline is implemented according to the paper An Improved Prediction System for Football a Match Result bu C. P. Igiri and E. O. Nwachukwu from 2014. The original features as described in the paper (except for few ones which are not available for the BSA) are used and the model is evaluated on the BSA dataset.

The authors used only one season for training, so I did the same. I used seasons 2017 for training season 2018 as validation set and the second half of season 2019 for testing. The validation set is used to select the best checkpoint of the training according to the validation accuracy.

In [ ]:
full_dataset = pd.read_csv('BRA.csv')
full_dataset.rename(columns = {'Home':'HomeTeam', 'Away': 'AwayTeam',
                               'HG': 'FTHG', 'AG': 'FTAG', 'Res': 'FTR',
                               'PH': 'B365H', 'PD': 'B365D',
                               'PA': 'B365A'}, inplace = True)
for season in full_dataset['Season'].unique():
  dataset = full_dataset[full_dataset['Season'] == season]
  dataset.to_csv('BSA_' + str(season)[-2:] + '.csv', index=False)

In [ ]:
X_train_win_loss, y_train = win_loss_functions.create_data(['BSA_17.csv'], skip_rounds = 6, return_names=True)
results_val_win_loss, matches_per_round = win_loss_functions.create_data_single('BSA_18.csv', ['BSA_17.csv'], skip_rounds = 6, return_names=True)
# Dates are returned as well for dividing testing season into slices
results_test_win_loss, matches_per_round = win_loss_functions.create_data_single('BSA_19.csv', ['BSA_17.csv', 'BSA_18.csv'],
                                return_dates=True, skip_rounds = 6, return_names=True)

Processing BSA_17.csv season file.


In [ ]:
# Arguments for getting the attacks are not needed
results_train_originals, matches_per_round = ann_functions.create_data_single('BSA_17.csv', None, None, None, None, include_stats=False)
results_val_originals, matches_per_round = ann_functions.create_data_single('BSA_18.csv', None, None, None, None, include_stats=False)
# Dates are returned as well for dividing testing season into slices
results_test_originals, matches_per_round = ann_functions.create_data_single('BSA_19.csv', None, None, None, None, include_stats=False,
                                  return_dates=True)
X_train_originals = results_train_originals.drop('FTR', axis=1)
y_train_originals = results_train_originals['FTR']

In [ ]:
# Concatenating the two feature sets together
X_train_mix = pd.concat([X_train_originals, X_train_win_loss], axis=1)
results_val_mix = pd.concat([results_val_originals, results_val_win_loss], axis=1)
results_test_mix = pd.concat([results_test_originals, results_test_win_loss], axis=1)
X_train_mix = X_train_mix.loc[:,~X_train_mix.columns.duplicated()].copy()
results_val_mix = results_val_mix.loc[:,~results_val_mix.columns.duplicated()].copy()
results_test_mix = results_test_mix.loc[:,~results_test_mix.columns.duplicated()].copy()

In [ ]:
A = pd.read_csv('A_BSA.csv')
H = pd.read_csv('H_BSA.csv')
A_before_threshold = pd.read_csv('A_BSA_before_threshold.csv')
H_before_threshold = pd.read_csv('H_BSA_before_threshold.csv')

In [ ]:
# Adding the feature vectors to the features
X_train = feature_vectors_functions.add_feature_vector(X_train_mix, A, H)
results_val = feature_vectors_functions.add_feature_vector(results_val_mix, A, H)
results_test = feature_vectors_functions.add_feature_vector(results_test_mix, A, H)

In [ ]:
X_val = results_val.drop('FTR', axis=1)
y_val = results_val['FTR']
# The classes need to go from 0 to 2 not from -1 to 1.
y_train += 1
y_val += 1

They didnt say what ANN architecture they used. I used 3 dense layers with 64, 32 and 16 neurons. I used dropout to overcome overfitting.

I always let the model train for 100 epochs and took the weights from the best epoch according to validation accuracy.

To use all the data available, the first half of the testing season was added to the training data.

In [ ]:
# Some rounds in the beginning are ignored, this is the correct index
# of the start of the second half of the season
start_test_index = 13 * matches_per_round

In [ ]:
X_test_to_append, y_test_to_append = testing_functions.prepare_test_to_append(results_test,
                                                                              start_test_index)
y_test_to_append += 1

In [ ]:
# Adding 1st half of testing season to the training data
X_train = pd.concat([X_train, X_test_to_append])
y_train = pd.concat([y_train, y_test_to_append])
# Rounds of the testing dataset
slices = testing_functions.get_slices(results_test, matches_per_round,
                                      start_test_index)
weighted_sum = 0
sum = 0
for slc in tqdm(slices):
  # Creating the model
  model = ann_functions.func_model((X_train.shape[1],))
  model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
  mc = tf.keras.callbacks.ModelCheckpoint('./weights_model.h5',
                                     monitor='val_accuracy',
                                     save_weights_only=True,
                                     save_best_only=True)
  X_test = slc.drop(['FTR', 'Date'], axis=1)
  y_test = slc['FTR']
  y_test += 1
  # Train the model
  history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=8,
                    validation_data=(X_val, y_val),
                    verbose = 0,
                    callbacks=[mc])
  # Load the best checkpoint
  model.load_weights('weights_model.h5')
  weighted_sum += (model.evaluate(X_test, y_test)[1] * len(y_test))
  sum += len(y_test)
  # Add the round to the training dataset
  X_train = pd.concat([X_train, X_test])
  y_train = pd.concat([y_train, y_test])
print('')
print(weighted_sum / sum)

  0%|          | 0/14 [00:00<?, ?it/s]

1/1 [==============================] - 0s 139ms/step - loss: 0.9598 - accuracy: 0.4545


  7%|▋         | 1/14 [00:21<04:38, 21.41s/it]

1/1 [==============================] - 0s 147ms/step - loss: 0.9709 - accuracy: 0.6429


 14%|█▍        | 2/14 [00:42<04:16, 21.39s/it]

...

1/1 [==============================] - 0s 149ms/step - loss: 0.9138 - accuracy: 0.6923


 93%|█████████▎| 13/14 [04:26<00:20, 20.63s/it]

1/1 [==============================] - 0s 149ms/step - loss: 1.0003 - accuracy: 0.4706


100%|██████████| 14/14 [04:47<00:00, 20.56s/it]


0.5052631680902682


The testing accuracy is 50.53%.

Experimenting with feature vectors extracted before applying the threshold.

In [ ]:
# y_train was changed, load it again
X_train_win_loss, y_train = win_loss_functions.create_data(['BSA_17.csv'], skip_rounds = 6, return_names=True)

Processing BSA_17.csv season file.


In [ ]:
# Adding the feature vectors to the features
X_train = feature_vectors_functions.add_feature_vector(X_train_mix, A_before_threshold, H_before_threshold)
results_val = feature_vectors_functions.add_feature_vector(results_val_mix, A_before_threshold, H_before_threshold)
results_test = feature_vectors_functions.add_feature_vector(results_test_mix, A_before_threshold, H_before_threshold)

In [ ]:
X_val = results_val.drop('FTR', axis=1)
y_val = results_val['FTR']
# The classes need to go from 0 to 2 not from -1 to 1.
y_train += 1
y_val += 1

In [ ]:
X_test_to_append, y_test_to_append = testing_functions.prepare_test_to_append(results_test,
                                                                              start_test_index)
y_test_to_append += 1

In [ ]:
# Adding 1st half of testing season to the training data
X_train = pd.concat([X_train, X_test_to_append])
y_train = pd.concat([y_train, y_test_to_append])
# Rounds of the testing dataset
slices = testing_functions.get_slices(results_test, matches_per_round,
                                      start_test_index)
weighted_sum = 0
sum = 0
for slc in tqdm(slices):
  # Creating the model
  model = ann_functions.func_model((X_train.shape[1],))
  model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
  mc = tf.keras.callbacks.ModelCheckpoint('./weights_model.h5',
                                     monitor='val_accuracy',
                                     save_weights_only=True,
                                     save_best_only=True)
  X_test = slc.drop(['FTR', 'Date'], axis=1)
  y_test = slc['FTR']
  y_test += 1
  # Train the model
  history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=8,
                    validation_data=(X_val, y_val),
                    verbose = 0,
                    callbacks=[mc])
  # Load the best checkpoint
  model.load_weights('weights_model.h5')
  weighted_sum += (model.evaluate(X_test, y_test)[1] * len(y_test))
  sum += len(y_test)
  # Add the round to the training dataset
  X_train = pd.concat([X_train, X_test])
  y_train = pd.concat([y_train, y_test])
print('')
print(weighted_sum / sum)

  0%|          | 0/14 [00:00<?, ?it/s]

1/1 [==============================] - 0s 124ms/step - loss: 1.0402 - accuracy: 0.3636


  7%|▋         | 1/14 [00:17<03:41, 17.04s/it]

1/1 [==============================] - 0s 122ms/step - loss: 0.9170 - accuracy: 0.7143


 14%|█▍        | 2/14 [00:38<03:54, 19.51s/it]

...

1/1 [==============================] - 0s 122ms/step - loss: 0.9679 - accuracy: 0.5385


 93%|█████████▎| 13/14 [04:17<00:20, 20.62s/it]

1/1 [==============================] - 0s 123ms/step - loss: 1.0627 - accuracy: 0.5294


100%|██████████| 14/14 [04:38<00:00, 19.90s/it]


0.5105263286515286


The testing accuracy is 51.05%.

Experimenting with feature vectors including home feature vectors for the away teams and away feature vectors for the home teams.

In [ ]:
# y_train was changed, load it again
X_train_win_loss, y_train = win_loss_functions.create_data(['BSA_17.csv'], skip_rounds = 6, return_names=True)

Processing BSA_17.csv season file.


In [ ]:
# Adding the feature vectors to the features
X_train = feature_vectors_functions.add_feature_vector(X_train_mix, A, H, include_all=True)
results_val = feature_vectors_functions.add_feature_vector(results_val_mix, A, H, include_all=True)
results_test = feature_vectors_functions.add_feature_vector(results_test_mix, A, H, include_all=True)

In [ ]:
X_val = results_val.drop('FTR', axis=1)
y_val = results_val['FTR']
# The classes need to go from 0 to 2 not from -1 to 1.
y_train += 1
y_val += 1

In [ ]:
X_test_to_append, y_test_to_append = testing_functions.prepare_test_to_append(results_test,
                                                                              start_test_index)
y_test_to_append += 1

In [ ]:
# Adding 1st half of testing season to the training data
X_train = pd.concat([X_train, X_test_to_append])
y_train = pd.concat([y_train, y_test_to_append])
# Rounds of the testing dataset
slices = testing_functions.get_slices(results_test, matches_per_round,
                                      start_test_index)
weighted_sum = 0
sum = 0
for slc in tqdm(slices):
  # Creating the model
  model = ann_functions.func_model((X_train.shape[1],))
  model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
  mc = tf.keras.callbacks.ModelCheckpoint('./weights_model.h5',
                                     monitor='val_accuracy',
                                     save_weights_only=True,
                                     save_best_only=True)
  X_test = slc.drop(['FTR', 'Date'], axis=1)
  y_test = slc['FTR']
  y_test += 1
  # Train the model
  history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=8,
                    validation_data=(X_val, y_val),
                    verbose = 0,
                    callbacks=[mc])
  # Load the best checkpoint
  model.load_weights('weights_model.h5')
  weighted_sum += (model.evaluate(X_test, y_test)[1] * len(y_test))
  sum += len(y_test)
  # Add the round to the training dataset
  X_train = pd.concat([X_train, X_test])
  y_train = pd.concat([y_train, y_test])
print('')
print(weighted_sum / sum)

  0%|          | 0/14 [00:00<?, ?it/s]

1/1 [==============================] - 0s 120ms/step - loss: 1.0219 - accuracy: 0.4545


  7%|▋         | 1/14 [00:15<03:17, 15.19s/it]

1/1 [==============================] - 0s 123ms/step - loss: 0.9654 - accuracy: 0.5714


 14%|█▍        | 2/14 [00:30<03:05, 15.50s/it]

...

1/1 [==============================] - 0s 117ms/step - loss: 0.9129 - accuracy: 0.6923


 93%|█████████▎| 13/14 [04:00<00:19, 19.36s/it]

1/1 [==============================] - 0s 121ms/step - loss: 0.9426 - accuracy: 0.5294


100%|██████████| 14/14 [04:21<00:00, 18.69s/it]


0.5000000131757636


The testing accuracy is 50%.

Experimenting with feature vectors extracted before applying the threshold and including home feature vectors for the away teams and away feature vectors for the home teams.

In [ ]:
# y_train was changed, load it again
X_train_win_loss, y_train = win_loss_functions.create_data(['BSA_17.csv'], skip_rounds = 6, return_names=True)

Processing BSA_17.csv season file.


In [ ]:
# Adding the feature vectors to the features
X_train = feature_vectors_functions.add_feature_vector(X_train_mix, A_before_threshold, H, include_all=True)
results_val = feature_vectors_functions.add_feature_vector(results_val_mix, A_before_threshold, H_before_threshold, include_all=True)
results_test = feature_vectors_functions.add_feature_vector(results_test_mix, A_before_threshold, H_before_threshold, include_all=True)

In [ ]:
X_val = results_val.drop('FTR', axis=1)
y_val = results_val['FTR']
# The classes need to go from 0 to 2 not from -1 to 1.
y_train += 1
y_val += 1

In [ ]:
X_test_to_append, y_test_to_append = testing_functions.prepare_test_to_append(results_test,
                                                                              start_test_index)
y_test_to_append += 1

In [ ]:
# Adding 1st half of testing season to the training data
X_train = pd.concat([X_train, X_test_to_append])
y_train = pd.concat([y_train, y_test_to_append])
# Rounds of the testing dataset
slices = testing_functions.get_slices(results_test, matches_per_round,
                                      start_test_index)
weighted_sum = 0
sum = 0
for slc in tqdm(slices):
  # Creating the model
  model = ann_functions.func_model((X_train.shape[1],))
  model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
  mc = tf.keras.callbacks.ModelCheckpoint('./weights_model.h5',
                                     monitor='val_accuracy',
                                     save_weights_only=True,
                                     save_best_only=True)
  X_test = slc.drop(['FTR', 'Date'], axis=1)
  y_test = slc['FTR']
  y_test += 1
  # Train the model
  history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=8,
                    validation_data=(X_val, y_val),
                    verbose = 0,
                    callbacks=[mc])
  # Load the best checkpoint
  model.load_weights('weights_model.h5')
  weighted_sum += (model.evaluate(X_test, y_test)[1] * len(y_test))
  sum += len(y_test)
  # Add the round to the training dataset
  X_train = pd.concat([X_train, X_test])
  y_train = pd.concat([y_train, y_test])
print('')
print(weighted_sum / sum)

  0%|          | 0/14 [00:00<?, ?it/s]

1/1 [==============================] - 0s 137ms/step - loss: 1.0369 - accuracy: 0.3636


  7%|▋         | 1/14 [00:17<03:44, 17.26s/it]

1/1 [==============================] - 0s 135ms/step - loss: 0.9962 - accuracy: 0.6429


 14%|█▍        | 2/14 [00:34<03:27, 17.31s/it]

...

1/1 [==============================] - 0s 217ms/step - loss: 0.9651 - accuracy: 0.5385


 93%|█████████▎| 13/14 [04:15<00:20, 20.50s/it]

1/1 [==============================] - 0s 138ms/step - loss: 0.9971 - accuracy: 0.5294


100%|██████████| 14/14 [04:38<00:00, 19.87s/it]


0.5000000112935117


The testing accuracy is 50%.